In [1]:
%pip install enchant

Note: you may need to restart the kernel to use updated packages.


# References

**Use pre-trained BERT in Mask Language to improve spell checking**
(I didn't think of that before)

- use a spell checker like **enchant** to mask mispelled words
- use **NLTK's part-of-speech tagging** to avoid masking uncommon names

https://web.archive.org/web/20220507023114/https://www.statestitle.com/resource/using-nlp-bert-to-improve-ocr-accuracy/

**for preprocessing**

https://aclanthology.org/2022.ltedi-1.44.pdf

In [3]:
from itertools import islice

import pkg_resources
from symspellpy import SymSpell, Verbosity

sym_spell = SymSpell()
dictionary_path = pkg_resources.resource_filename(
    "symspellpy", "frequency_dictionary_en_82_765.txt"
)
sym_spell.load_dictionary(dictionary_path, 0, 1)

# Print out first 5 elements to demonstrate that dictionary is
# successfully loaded
print(list(islice(sym_spell.words.items(), 5)))

[('the', 23135851162), ('of', 13151942776), ('and', 12997637966), ('to', 12136980858), ('a', 9081174698)]


In [4]:
# lookup suggestions for single-word input strings
input_term = (
    "at a grand opening of a packed sports bar during the pacquiao-algieri fight chaos erupted when a patron was denied reentry by the bouncer angry the man left returned with a gun and intentionally shot the bouncer in the head the bar descended into panic as people hit the floor or fled my friends and i waited until it was safe then left stepping over the dead bouncer"
)
# max edit distance per lookup
# (max_edit_distance_lookup <= max_dictionary_edit_distance)
suggestions = sym_spell.lookup_compound(input_term, max_edit_distance=2)
# display suggestion term, edit distance, and term frequency
for suggestion in suggestions:
    print(suggestion)

at a grand opening of a packed sports bar during the a acquire algiers fight chaos erupted when a patron was denied reentry by the bouncer angry the man left returned with a gun and intentionally shot the bouncer in the head the bar descended into panic as people hit the floor or fled my friends and i waited until it was safe then left stepping over the dead bouncer, 6, 0


In [9]:
# import torch
# from pytorch_pretrained_bert import BertTokenizer, BertForMaskedLM
import re
import nltk
from enchant.checker import SpellChecker
from difflib import SequenceMatcher
filename = './sample6.png'
text_original = "Hillo werld oh no yes this is wrong spelling by Michaella" # get text from dataset
print (text_original)

ModuleNotFoundError: No module named 'enchant.checker'

In [1]:
%pip install nltk

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.5/1.5 MB 1.1 MB/s eta 0:00:00a 0:00:010m
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 776.5/776.5 KB 1.2 MB/s eta 0:00:00a 0:00:01
  Using cached click-8.1.7-py3-none-any.whl (97 kB)
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 78.4/78.4 KB 1.5 MB/s eta 0:00:00a 0:00:01
Note: you may need to restart the kernel to use updated packages.


In [ ]:
# cleanup text
rep = { '\n': ' ', '\\': ' ', '\"': '"', '-': ' ', '"': ' " ', 
        '"': ' " ', '"': ' " ', ',':' , ', '.':' . ', '!':' ! ', 
        '?':' ? ', "n't": " not" , "'ll": " will", '*':' * ', 
        '(': ' ( ', ')': ' ) ', "s'": "s '"}
rep = dict((re.escape(k), v) for k, v in rep.items()) 
pattern = re.compile("|".join(rep.keys()))
text = pattern.sub(lambda m: rep[re.escape(m.group(0))], text)
def get_personslist(text):
    personslist=[]
    for sent in nltk.sent_tokenize(text):
        for chunk in nltk.ne_chunk(nltk.pos_tag(nltk.word_tokenize(sent))):
            if isinstance(chunk, nltk.tree.Tree) and chunk.label() == 'PERSON':
                personslist.insert(0, (chunk.leaves()[0][0]))
    return list(set(personslist))
personslist = get_personslist(text)
ignorewords = personslist + ["!", ",", ".", "\"", "?", '(', ')', '*', '\'']
# using enchant.checker.SpellChecker, identify incorrect words
d = SpellChecker("en_US")
words = text.split()
incorrectwords = [w for w in words if not d.check(w) and w not in ignorewords]
# using enchant.checker.SpellChecker, get suggested replacements
suggestedwords = [d.suggest(w) for w in incorrectwords]
# replace incorrect words with [MASK]
for w in incorrectwords:
    text = text.replace(w, '[MASK]')
    text_original = text_original.replace(w, '[MASK]')
    
print(text)# cleanup text

In [11]:
import nltk

NLTK - pos tagging

In [14]:
text = "at a grand opening of a packed sports bar during the pacquiao-algieri fight chaos erupted when a patron was denied reentry by the bouncer angry the man left returned with a gun and intentionally shot the bouncer in the head the bar descended into panic as people hit the floor or fled my friends and i waited until it was safe then left stepping over the dead bouncerat a grand opening of a packed sports bar during the pacquiao-algieri fight chaos erupted when a patron was denied reentry by the bouncer angry the man left returned with a gun and intentionally shot the bouncer in the head the bar descended into panic as people hit the floor or fled my friends and i waited until it was safe then left stepping over the dead bouncer" 
nltk.word_tokenize(text)

LookupError: 
**********************************************************************
  Resource [93mpunkt_tab[0m not found.
  Please use the NLTK Downloader to obtain the resource:

  [31m>>> import nltk
  >>> nltk.download('punkt_tab')
  [0m
  For more information see: https://www.nltk.org/data.html

  Attempted to load [93mtokenizers/punkt_tab/english/[0m

  Searched in:
    - '/home/syke/nltk_data'
    - '/media/syke/Colorful/Github/xlnet/.venv/nltk_data'
    - '/media/syke/Colorful/Github/xlnet/.venv/share/nltk_data'
    - '/media/syke/Colorful/Github/xlnet/.venv/lib/nltk_data'
    - '/usr/share/nltk_data'
    - '/usr/local/share/nltk_data'
    - '/usr/lib/nltk_data'
    - '/usr/local/lib/nltk_data'
**********************************************************************


In [13]:
nltk.download('all')

[nltk_data] Downloading collection 'all'
[nltk_data]    | 
[nltk_data]    | Downloading package abc to /home/syke/nltk_data...
[nltk_data]    |   Package abc is already up-to-date!
[nltk_data]    | Downloading package alpino to /home/syke/nltk_data...
[nltk_data]    |   Package alpino is already up-to-date!
[nltk_data]    | Downloading package averaged_perceptron_tagger to
[nltk_data]    |     /home/syke/nltk_data...
[nltk_data]    |   Package averaged_perceptron_tagger is already up-
[nltk_data]    |       to-date!
[nltk_data]    | Downloading package averaged_perceptron_tagger_eng to
[nltk_data]    |     /home/syke/nltk_data...
[nltk_data]    |   Package averaged_perceptron_tagger_eng is already
[nltk_data]    |       up-to-date!
[nltk_data]    | Downloading package averaged_perceptron_tagger_ru to
[nltk_data]    |     /home/syke/nltk_data...
[nltk_data]    |   Package averaged_perceptron_tagger_ru is already
[nltk_data]    |       up-to-date!
[nltk_data]    | Downloading package ave

KeyboardInterrupt: 